## Load Database

In [2]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine("sqlite:///../plantation.db")

query = "SELECT * FROM plantation LIMIT 5"

pd.read_sql(query, engine)


,date,region,production_ton,operational_cost,sales_revenue,inventory_stock,profit
0,2024-01-01,Sulawesi,183,7977.0,17231,336,9254
1,2024-01-02,Sumatra,125,5602.0,7009,380,1407
2,2024-01-03,Sulawesi,103,6134.0,7260,458,1126
3,2024-01-04,Sulawesi,193,5522.0,13038,300,7516
4,2024-01-05,Sumatra,111,6268.0,16865,345,10597


## Bussines Question

*Q1 — Profit rata-rata per region*

In [3]:
query = """
SELECT region, AVG(profit) AS avg_profit
FROM plantation
GROUP BY region
ORDER BY avg_profit DESC;
"""

pd.read_sql(query, engine)

,region,avg_profit
0,Sulawesi,8437.748031
1,Sumatra,8211.611111
2,Kalimantan,8210.383929


*Q2 — Total profit per bulan*

In [4]:
query = """
SELECT strftime('%Y-%m', date) AS month,
SUM(profit) AS total_profit
FROM plantation
GROUP BY month
ORDER BY month;
"""

pd.read_sql(query, engine)


,month,total_profit
0,2024-01,253254
1,2024-02,227301
2,2024-03,271630
3,2024-04,233314
4,2024-05,277462
5,2024-06,233971
6,2024-07,255113
7,2024-08,230385
8,2024-09,280384
9,2024-10,279295


*Region ranking berdasarkan profit*

In [5]:
query = """
SELECT region,
SUM(profit) AS total_profit,
RANK() OVER (ORDER BY SUM(profit) DESC) AS rank
FROM plantation
GROUP BY region;
"""

pd.read_sql(query, engine)

,region,total_profit,rank
0,Sulawesi,1071594,1
1,Sumatra,1034663,2
2,Kalimantan,919563,3


## Insight